In [12]:
import os, sys
import torch
import argparse
from scipy import ndimage

In [3]:
#!export PYTHONPATH=$PYTHONPATH:/data1/practical-sose23/castellvi/castellvi_prediction/bids
sys.path.append('/data1/practical-sose23/castellvi/castellvi_prediction/bids')

In [4]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from utils._prepare_data import DataHandler
from modules.ResNetModule import ResNet
from modules.VerSeDataModule import VerSeDataModule
from modules.DenseNetModule import DenseNet
from modules.DenseNetModule_v2 import DenseNetV2
from utils.environment_settings import env_settings
import torch
from torch import nn

####################################
/u/home/mamar/.conda/envs/env-arda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  File "/u/home/mamar/.conda/envs/env-arda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/u/home/mamar/.conda/envs/env-arda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/u/home/mamar/.conda/envs/env-arda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/u/home/mamar/.conda/envs/env-arda/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/u/home/mamar/.conda/envs/env-arda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
    self.io_

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(env_settings.CUDA_VISIBLE_DEVICES)
if torch.cuda.is_available():
    print('Running on GPU #' + str(torch.cuda.current_device()))

Running on GPU #0


In [6]:
parser = argparse.ArgumentParser(description='Evaluation settings')

parser = argparse.ArgumentParser(description='Evaluation settings')
parser.add_argument('--data_root', nargs='+', default=[str(os.path.join(env_settings.DATA, 'dataset-verse19')),
                                                        str(os.path.join(env_settings.DATA, 'dataset-verse20')),
                                                        str(os.path.join(env_settings.DATA, 'dataset-tri'))])
parser.add_argument('--data_types', nargs='+', default=['rawdata', 'derivatives'])
parser.add_argument('--img_types', nargs='+', default=['ct', 'subreg', 'cortex'])
parser.add_argument('--master_list', default= str(os.path.join(env_settings.ROOT, 'src/dataset/Castellvi_list_Final_Split.xlsx')))
parser.add_argument('--classification_type', default='right_side')
parser.add_argument('--castellvi_classes', nargs='+', default=['1a', '1b', '2a', '2b', '3a', '3b', '4', '0'])
parser.add_argument('--model', default='densenet')
parser.add_argument('--phase', default='train')
parser.add_argument('--scheduler', default='ReduceLROnPlateau')
parser.add_argument('--optimizer', default='AdamW')
parser.add_argument('--learning_rate', type=float, default=0.0001)
parser.add_argument('--weight_decay', type=float, default=0.0001)
parser.add_argument('--total_iterations', type=int, default=100)
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--accumulate_grad_batches', type=int, default=1)
parser.add_argument('--num_workers', type=int, default=8)
parser.add_argument('--save_intervals', type=int, default=10)
parser.add_argument('--n_epochs', type=int, default=100)
parser.add_argument('--resume_path', default='')
parser.add_argument('--experiments', default=env_settings.EXPERIMENTS)
parser.add_argument('--gpu_id', default='0')
parser.add_argument('--n_devices', type=int, default=1)
parser.add_argument('--manual_seed', type=int, default=1)
parser.add_argument('--num_classes', type=int, default=3)
parser.add_argument('--port', type=int, default=2023)
parser.add_argument('--model_type', type=str, default='')
parser.add_argument('--dropout_prob', type=float, default=0.3)


parser.add_argument('--rotate_range', type=int, default=10)
parser.add_argument('--shear_range', type=float, default=0.2)
parser.add_argument('--translate_range', type=float, default=0.15)
parser.add_argument('--scale_range', nargs='+', default=[0.9, 1.1])
parser.add_argument('--aug_prob', type=float, default=0.5)

parser.add_argument('--use_seg', default=True)
parser.add_argument('--no_cuda', default=False)
parser.add_argument('--weighted_sample', default=True)
parser.add_argument('--weighted_loss', default=False)
parser.add_argument('--flip_all', default=True)
parser.add_argument('--cross_validation',default=False)
parser.add_argument('--use_bin_seg', default=True)
parser.add_argument('--use_zero_out',default=True)

parser.add_argument('--elastic_transform', default=True)
parser.add_argument('--seg_comparison', default=False)

parser.add_argument('--version_no', type=int, default=10)
parser.add_argument('--dataset', nargs='+', default=['verse', 'tri'])
parser.add_argument('--eval_type', type=str, default='test')

params, unknown = parser.parse_known_args()


In [7]:
base_experiment = params.experiments + '/baseline_models/' + params.model
ckpt_path = base_experiment + '/best_models/version_' + str(params.version_no)
best_model= os.listdir(ckpt_path)[0]
best_model_path = os.path.join(ckpt_path, best_model)

In [15]:
from eval import Eval


In [16]:
evaluator = Eval(params)
best_model= os.listdir(ckpt_path)[0]
best_model_path = os.path.join(ckpt_path, best_model)

In [17]:
model = evaluator.load_model(best_model_path, params)

DenseNet loaded with 3 classes, data_size (96, 78, 78) and 1 channel


In [18]:
processor = evaluator.get_processor()

[!] Unknown format seg-ano in file sub-verse649_dir-sag_seg-ano.nii.gz
[!] Unknown format iso-ctd in file sub-verse616_dir-iso_iso-ctd.json
[!] "template" is not a valid key/value pair. Expected "KEY-VALUE" in sub-verse519_template_sacrum_msk.nii.gz
[!] "sacrum" is not a valid key/value pair. Expected "KEY-VALUE" in sub-verse519_template_sacrum_msk.nii.gz
[!] Unknown format subreg in file verse559_CT-sag_seg-ano_subreg.nii.gz
[!] "verse559" is not a valid key/value pair. Expected "KEY-VALUE" in verse559_CT-sag_seg-ano_subreg.nii.gz
[!] "verse559" is not a valid key/value pair. Expected "KEY-VALUE" in verse559_CT-sag_seg-ano.nii.gz
[!] "verse549" is not a valid key/value pair. Expected "KEY-VALUE" in verse549_CT-iso_seg-ano.nii.gz
[!] Unknown format ce-ar in file sub-tri106_ce-ar.nii.gz
[!] Unknown format ce-pv in file sub-tri066_ce-pv.nii.gz
[!] Unknown format ce-ne in file sub-tri067_ce-ne.nii.gz
[!] Unknown format ce-late in file sub-tri023_ce-late.nii.gz
[!] Unknown format stat in f

In [19]:
test_subjects = evaluator.get_test_subjects()

eval type is not all


In [20]:
records = evaluator.get_records(processor, test_subjects)

lenght of the test dataset: 73


In [23]:
# How to get output from the model
# get one sample record
sample_record = records[0]
model_inputs = evaluator.process_input(processor, sample_record)
input_img = model_inputs['original'] # you can also get flipped one by model_inputs['flipped']
output_class, output_prob = evaluator.process_output(model, input_img)


input shape: torch.Size([1, 1, 96, 78, 78])


In [24]:
output_class

'0'